In [1]:
%load_ext autoreload
%autoreload 2

from gwak.train.dataloader import SignalDataloader
import matplotlib.pyplot as plt
import numpy as np
import torch


device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'

/home/jack.redepenning/miniconda3/envs/gwak/lib/python3.11/site-packages/lightning/fabric/__init__.py:41: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.


In [2]:
data_dir = "/home/aframe/dev/o4_llpic/amplfi/hl/data"
# data_dir = "/home/hongyin.chen/Data/GWAK_data/gwak_inspect_4K"
sample_rate = 4096
kernel_length = 0.5
psd_length = 64
fduration = 1
fftlength = 2
batch_size = 2
batches_per_epoch = 50
num_workers = 2
data_saving_file = None
duration = fduration + kernel_length

from gwak.data.prior import SineGaussianBBC, LAL_BBHPrior, GaussianBBC, CuspBBC, KinkBBC, KinkkinkBBC, WhiteNoiseBurstBBC
from ml4gw.waveforms import SineGaussian, IMRPhenomPv2, Gaussian, GenerateString, WhiteNoiseBurst

signal_classes = [
    "SineGaussian",
    "BBH",
    "Gaussian",
    "Cusp",
    "Kink",
    "KinkKink",
    "WhiteNoiseBurst",
    "Background"
]
priors = [
    SineGaussianBBC(),
    LAL_BBHPrior(),
    GaussianBBC(),
    CuspBBC(),
    KinkBBC(),
    KinkkinkBBC(),
    WhiteNoiseBurstBBC(),
    None
]
waveforms = [
    SineGaussian(
        sample_rate=sample_rate,
        duration=duration
    ),
    IMRPhenomPv2(),
    Gaussian(
        sample_rate=sample_rate,
        duration=duration
    ),
    GenerateString(
        sample_rate=sample_rate
    ),
    GenerateString(
        sample_rate=sample_rate
    ),
    GenerateString(
        sample_rate=sample_rate
    ),
    WhiteNoiseBurst(
        sample_rate=sample_rate,
        duration=duration
    ),
    None
]
extra_kwargs = [
    None,
    {"ringdown_duration":0.9},
    None,
    None,
    None,
    None,
    None,
    None
]

In [3]:
loader = SignalDataloader(signal_classes,
    priors,
    waveforms,
    extra_kwargs,
    data_dir=data_dir,
    sample_rate=sample_rate,
    kernel_length=kernel_length,
    psd_length=psd_length,
    fduration=fduration,
    fftlength=fftlength,
    batch_size=batch_size,
    batches_per_epoch=batches_per_epoch,
    num_workers=num_workers,
    data_saving_file=data_saving_file,
    snr_prior=None
)
test_loader = loader.test_dataloader()

You passed a directory with a test/ directory but no val/, setting val set = test set
ifos are ['H1', 'L1']
data dir is /home/aframe/dev/o4_llpic/amplfi/hl/data


In [5]:
next(iter(test_loader))[0].shape

torch.Size([2, 2, 268288])

In [9]:
from tqdm import tqdm

for batch in tqdm(test_loader):
    batch = batch[0].T
    # print(batch.T.shape)
    waveforms, params, ras, decs, phics = loader.generate_waveforms(batch.shape[0])

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:03<?, ?it/s]


RuntimeError: MKL FFT error: Intel oneMKL DFTI ERROR: Inconsistent configuration parameters